In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from pathlib import Path
import os
import pickle
from tqdm import tqdm
from config import model_config

from sklearn.ensemble import (
	RandomForestClassifier
)
from sklearn.feature_selection import(
	RFECV, SequentialFeatureSelector
)
from sklearn.linear_model import (
	LinearRegression, LogisticRegression,
)
from sklearn.metrics import (
	confusion_matrix, classification_report, f1_score,
	roc_curve, roc_auc_score
)
from sklearn.model_selection import (
	train_test_split, RandomizedSearchCV, GridSearchCV, 
	cross_val_score, cross_val_predict, KFold,
)
from sklearn.pipeline import (
	Pipeline
)
from sklearn.preprocessing import (
	LabelEncoder, OneHotEncoder, StandardScaler,
	RobustScaler, QuantileTransformer,
)
import statsmodels.api as sm

from regression import reg

In [2]:
SEED = 123
TEST_SIZE = 0.25
CV_FOLDS = 10

HEATMAP_COLORS = sns.diverging_palette(h_neg=359, h_pos=250, as_cmap=True)

# Import Data

In [3]:
with open(Path('../data/classification_data.pkl'), 'rb') as f:
	data = pickle.load(f)
	
X = data.get('X')
y = data.get('y')
body_features = data.get('body_features')
cardio_features = data.get('cardio_features')

print(X.shape)
print(y.shape)
print(body_features)
print(cardio_features)

X.head()

(43, 17)
(43, 1)
['density_visceral_fat', 'mass_visceral_fat', 'density_intermuscular_fat', 'density_muscle', 'density_bone', 'bmi']
['emphysema_volume_950hu', 'extrapulmonary_vein_volume', 'artery_vein_ratio', 'bv10', 'pb_larger_10', 'pv_a', 'heart_volume', 'airway_ratio', 'ild_volume']


,density_visceral_fat,mass_visceral_fat,density_intermuscular_fat,density_muscle,density_bone,bmi,emphysema_volume_950hu,extrapulmonary_vein_volume,artery_vein_ratio,bv10,pb_larger_10,pv_a,heart_volume,airway_ratio,ild_volume,age,gender_cl_Male
PE1_0,-0.664818,-1.165809,-0.147435,-0.256701,-1.777867,0.203818,-0.106865,2.137818,-1.150427,0.494738,1.912364,1.263457,1.782875,-0.309816,-0.721954,-0.014636,1.0
PE12_0,-0.041982,1.031574,-0.799236,-0.453552,-0.626438,-0.164653,-0.476252,0.372377,0.021395,0.979777,0.403948,1.315332,0.195270,0.391257,1.080036,-0.282771,1.0
PE14_0,1.297917,-0.722682,1.549786,0.317693,-0.270970,-0.478136,1.156899,0.394634,-0.157870,0.451135,0.093329,0.009487,0.217828,-0.568843,-1.825349,-1.199497,0.0
PE15_0,1.177249,-1.684023,1.401407,1.052018,0.819345,-1.881309,-0.465319,-0.547764,0.329446,0.485155,-0.773213,0.272429,-1.091411,0.044701,-0.201688,-2.011256,0.0
PE16_0,-2.262411,1.481516,-1.468937,-0.354760,1.241557,1.294754,-0.753382,-0.176271,-1.989854,-1.067518,0.076410,-0.024398,-0.798537,1.291209,1.186939,-1.006231,0.0


In [4]:
y.head()

,resolved_pe
PE1_0,1.0
PE12_0,0.0
PE14_0,0.0
PE15_0,0.0
PE16_0,0.0


In [5]:
y.value_counts(dropna=False)

resolved_pe
0.0            28
1.0            15
dtype: int64

# Logit Regression

In [6]:
X_temp = sm.add_constant(X.loc[:, ['heart_volume', 'age']])
model = sm.Logit(y, X_temp).fit(cov_type='HC3', disp=False)
model.pvalues['heart_volume']

0.0678289785659096

In [7]:
def combine_feat_and_controls(feat, controls=None):
	all_feat = list()
	all_feat.append(feat)
	if controls is not None:
		for control in controls:
			all_feat.append(control)
	return all_feat

def logit_regression(X, y, feat=None, controls=None):
	all_feat = combine_feat_and_controls(feat, controls)

	# Fit statsmodels model for pvalues
	X_temp = sm.add_constant(X.loc[:, all_feat])
	model = sm.Logit(y, X_temp).fit(cov_type='HC3', disp=False)
	
	# Fit sklearn model for cross validation
	model_for_cv = LogisticRegression(random_state=SEED, max_iter=10_000, penalty=None, solver='lbfgs')

	logit_probs = model.predict()
	fpr, tpr, thresholds = roc_curve(y, logit_probs)
	auc = roc_auc_score(y, logit_probs)
	cv_auc = np.mean(cross_val_score(model_for_cv, X_temp, y, scoring='roc_auc', cv=CV_FOLDS, n_jobs=-1))

	return (
		model.params, 
		model.pvalues,
		fpr,
		tpr,
		thresholds,
		auc,
		cv_auc
	)

def logit_regressions(feature_pool, controls):
	logit_coefs = dict()
	logit_pvals = dict()
	logit_auc = dict()
	logit_cv_auc = dict()

	for feature in feature_pool:
		(
			coef, pvalues,
			fpr, tpr, thresholds, 
			auc, cv_auc
		) = logit_regression(X, y, feature, controls)

		logit_coefs[feature] = coef[feature]
		logit_pvals[feature] = pvalues[feature]
		logit_auc[feature] = auc
		logit_cv_auc[feature] = cv_auc

	if controls is None:
		controls_str = 'None'
	else:
		controls_str = ', '.join(controls)

	logit_results_df = pd.DataFrame(
		{
			'coef': logit_coefs,
			'pvalue': logit_pvals,
			'auc': logit_auc,
			'cv_auc': logit_cv_auc,
			'controls': controls_str,
		}
	)
	return logit_results_df

def plot_roc(fpr, tpr, auc, custom_title=None):
	fig, ax = plt.subplots()
	ax.plot([0, 1], ls='--', c='r')
	ax.plot([0, 0], [1, 0], c='0.8')
	ax.plot([1, 0], [1, 1], c='0.8')
	ax.plot(fpr, tpr, label=f'ROC Curve (AUC={auc:.3})')
	if custom_title is not None:
		ax.set_title(f'Y=Resolved PE, {custom_title}', fontsize=16)
	else:
		ax.set_title(f'Y=Resolved PE', fontsize=16)
	ax.set_xlabel('False Positive Rate')
	ax.set_ylabel('True Positive Rate')
	plt.legend(fontsize=14)
	plt.show()
	
def select_features_REFCV(model, scoring, features, target):
	feature_selection_results = dict()
	selector = RFECV(
		estimator=model,
		step=1,
		scoring=scoring,
		cv=CV_FOLDS
	)
	selector.fit(X[features], np.array(y).ravel())
	feature_selection_results[target] = list(selector.get_feature_names_out())
	return feature_selection_results

def select_features_SFS(model, scoring, features, target, direction):
	feature_selection_results = dict()
	selector = SequentialFeatureSelector(
		estimator=model,
		n_features_to_select='auto',
		tol=0.01,
		scoring=scoring,
		direction=direction,
		cv=10
	)
	selector.fit(X[features], np.array(y).ravel())
	feature_selection_results[target] = list(selector.get_feature_names_out())
	return feature_selection_results

def evaluate_multivariable_logit(feature_selection_results):
	regression_df = pd.DataFrame()
	target = 'resolved_pe'
	x_vars = feature_selection_results[target]
	X_temp = X.loc[:, x_vars]
	y_temp = y.loc[:, target]

	# Fit model (statsmodels, for p-values)
	model_sm = sm.Logit(y_temp, sm.add_constant(X_temp)).fit(cov_type='HC3', disp=False)

	# Fit models (sklearn, for CV scores)
	model_sk = LogisticRegression(random_state=SEED, max_iter=10_000, penalty=None, solver='lbfgs')
	model_sk_l2 = LogisticRegression(random_state=SEED, max_iter=10_000, penalty='l2', solver='liblinear')
	model_sk_l1 = LogisticRegression(random_state=SEED, max_iter=10_000, penalty='l1', solver='saga')
	model_sk_elastic = LogisticRegression(random_state=SEED, max_iter=10_000, penalty='elasticnet', solver='saga', l1_ratio=0.5)

	# Get score for simple model
	model_sk.fit(X_temp, np.array(y_temp).ravel())
	logit_probs = model_sk.predict_proba(X_temp)
	fpr, tpr, thresholds = roc_curve(np.array(y_temp).ravel(), logit_probs[:, 1])
	model_sk_auc = roc_auc_score(np.array(y_temp).ravel(), logit_probs[:, 1])

	# Get cross validation scores for variously regularized models
	model_sk_cv_auc = cross_val_score(model_sk, X_temp, y_temp, scoring='roc_auc', cv=CV_FOLDS, n_jobs=-1)
	model_sk_l2_cv_auc = cross_val_score(model_sk_l2, X_temp, y_temp, scoring='roc_auc', cv=CV_FOLDS, n_jobs=-1)
	model_sk_l1_cv_auc = cross_val_score(model_sk_l1, X_temp, y_temp, scoring='roc_auc', cv=CV_FOLDS, n_jobs=-1)
	model_sk_elastic_cv_auc = cross_val_score(model_sk_elastic, X_temp, y_temp, scoring='roc_auc', cv=CV_FOLDS, n_jobs=-1)

	# Store metrics in a df
	metrics_df = pd.DataFrame(
		[
			[np.mean(model_sk_auc), np.nan],
			[np.mean(model_sk_cv_auc), np.std(model_sk_cv_auc)],
			[np.mean(model_sk_l2_cv_auc), np.std(model_sk_l2_cv_auc)],
			[np.mean(model_sk_l1_cv_auc), np.std(model_sk_l1_cv_auc)],
			[np.mean(model_sk_elastic_cv_auc), np.std(model_sk_elastic_cv_auc)],
		],
		columns=['Mean AUC', 'Std AUC'],
		index=['All Data', 'No Regularization, CV-10', 'L2 Regularization, CV-10', 'L1 Regularization, CV-10', 'ElasticNet Regularization, CV-10']
	)

	# Store results in df
	res = pd.DataFrame({
		'coef': model_sm.params,
		'pval': model_sm.pvalues,
		'nobs': model_sm.nobs,
	})
	res['signif'] = res['pval'].apply(reg.add_significance)

	# Create new index
	res.index = pd.MultiIndex.from_tuples(
		list(zip([target for _ in range(res.shape[0])], res.index)),
		names=('y', 'X')
	)
	res.index.names

	# Add to final df
	regression_df = pd.concat((regression_df, res), axis=0)
	######################################################################################
	# Show results
	print("="*80)
	print(metrics_df)
	print("="*80)
	print("="*80)
	print(regression_df)
	print("="*80)
	print("\n")
	######################################################################################
	return dict(
		evaluation_metrics = metrics_df,
		regression_results = regression_df
	)



## Univariate Analysis

### Body Composition

In [8]:
# Get logit results
logit_body_dfs = list()
logit_body_dfs.append(logit_regressions(body_features, controls=None))
logit_body_dfs.append(logit_regressions(body_features, controls=['age']))
logit_body_dfs.append(logit_regressions(body_features, controls=['gender_cl_Male']))
logit_body_dfs.append(logit_regressions(body_features, controls=['gender_cl_Male', 'age']))

# Combine and reset index
all_logit_body_dfs = pd.concat(logit_body_dfs, axis=0)
all_logit_body_dfs['x_var'] = all_logit_body_dfs.index
all_logit_body_dfs.index = all_logit_body_dfs.index + ' (' + all_logit_body_dfs['controls'] + ')'

# Add significance
all_logit_body_dfs['signif'] = all_logit_body_dfs['pvalue'].apply(reg.add_significance)

# Export and show snippet
# all_logit_body_dfs.to_csv('../output/regressions/logit_body.csv')
all_logit_body_dfs.tail()

,coef,pvalue,auc,cv_auc,controls,x_var,signif
"mass_visceral_fat (gender_cl_Male, age)",-0.232165,0.604739,0.804762,0.675000,"gender_cl_Male, age",mass_visceral_fat,
"density_intermuscular_fat (gender_cl_Male, age)",0.055540,0.856964,0.795238,0.708333,"gender_cl_Male, age",density_intermuscular_fat,
"density_muscle (gender_cl_Male, age)",-0.104961,0.854202,0.792857,0.741667,"gender_cl_Male, age",density_muscle,
"density_bone (gender_cl_Male, age)",-0.349649,0.375176,0.807143,0.725000,"gender_cl_Male, age",density_bone,
"bmi (gender_cl_Male, age)",-0.010298,0.977295,0.790476,0.683333,"gender_cl_Male, age",bmi,


### Cardiopulmonary

In [9]:
# Get logit results
logit_cardio_dfs = list()
logit_cardio_dfs.append(logit_regressions(cardio_features, controls=None))
logit_cardio_dfs.append(logit_regressions(cardio_features, controls=['age']))
logit_cardio_dfs.append(logit_regressions(cardio_features, controls=['gender_cl_Male']))
logit_cardio_dfs.append(logit_regressions(cardio_features, controls=['gender_cl_Male', 'age']))

# Combine and reset index
all_logit_cardio_dfs = pd.concat(logit_cardio_dfs, axis=0)
all_logit_cardio_dfs['x_var'] = all_logit_cardio_dfs.index
all_logit_cardio_dfs.index = all_logit_cardio_dfs.index + ' (' + all_logit_cardio_dfs['controls'] + ')'

# Add significance
all_logit_cardio_dfs['signif'] = all_logit_cardio_dfs['pvalue'].apply(reg.add_significance)

# Export and show snippet
# all_logit_cardio_dfs.to_csv('../output/regressions/logit_cardio.csv')
all_logit_cardio_dfs.tail()

,coef,pvalue,auc,cv_auc,controls,x_var,signif
"pb_larger_10 (gender_cl_Male, age)",-0.155669,0.627738,0.790476,0.725000,"gender_cl_Male, age",pb_larger_10,
"pv_a (gender_cl_Male, age)",1.057619,0.084248,0.840476,0.750000,"gender_cl_Male, age",pv_a,
"heart_volume (gender_cl_Male, age)",0.322187,0.458636,0.802381,0.708333,"gender_cl_Male, age",heart_volume,
"airway_ratio (gender_cl_Male, age)",0.205298,0.646258,0.802381,0.700000,"gender_cl_Male, age",airway_ratio,
"ild_volume (gender_cl_Male, age)",0.348984,0.419294,0.811905,0.775000,"gender_cl_Male, age",ild_volume,


### Export results

In [10]:
logit_results = pd.concat([all_logit_body_dfs, all_logit_cardio_dfs], axis=0)
logit_results.to_csv('../output/regressions/logit_body_cardio.csv')

## Multivariable Analysis

In [11]:
# Define model for CV feat selection
logit = LogisticRegression(random_state=SEED, max_iter=10_000, penalty=None, solver='lbfgs')
logit_target = 'resolved_pe'

In [12]:
# List to store results
multivariable_results = dict()

### Recursive Feature Selection

In [13]:
# Cardio
refcv_cardio_features = select_features_REFCV(model=logit, scoring='roc_auc', features=cardio_features, target=logit_target)
print(refcv_cardio_features)

# Body
refcv_body_features = select_features_REFCV(model=logit, scoring='roc_auc', features=body_features, target=logit_target)
print(refcv_body_features)

{'resolved_pe': ['artery_vein_ratio', 'heart_volume', 'airway_ratio']}
{'resolved_pe': ['density_visceral_fat', 'density_muscle', 'density_bone', 'bmi']}


In [14]:
refcv_cardio = evaluate_multivariable_logit(refcv_cardio_features)
refcv_body = evaluate_multivariable_logit(refcv_body_features)

multivariable_results['refcv_cardio'] = refcv_cardio
multivariable_results['refcv_body'] = refcv_body

                                  Mean AUC   Std AUC
All Data                          0.752381       NaN
No Regularization, CV-10          0.700000  0.369309
L2 Regularization, CV-10          0.700000  0.369309
L1 Regularization, CV-10          0.641667  0.357557
ElasticNet Regularization, CV-10  0.641667  0.357557
                                   coef      pval  nobs signif
y           X                                                 
resolved_pe const             -0.637891  0.105654    43       
            artery_vein_ratio  0.672336  0.083340    43       
            heart_volume       0.727707  0.036744    43      *
            airway_ratio       0.841185  0.050088    43       


                                  Mean AUC   Std AUC
All Data                          0.773810       NaN
No Regularization, CV-10          0.766667  0.249444
L2 Regularization, CV-10          0.766667  0.260342
L1 Regularization, CV-10          0.733333  0.270801
ElasticNet Regularization, CV-10  0.7

### Sequential Feature Selection (Backward)

In [15]:
# Cardio
sfs_bwd_cardio_features = select_features_SFS(model=logit, scoring='roc_auc', features=cardio_features, target=logit_target, direction='backward')
print(sfs_bwd_cardio_features)

# Body
sfs_bwd_body_features = select_features_SFS(model=logit, scoring='roc_auc', features=body_features, target=logit_target, direction='backward')
print(sfs_bwd_body_features)

{'resolved_pe': ['extrapulmonary_vein_volume', 'artery_vein_ratio', 'bv10', 'pv_a', 'heart_volume', 'airway_ratio', 'ild_volume']}
{'resolved_pe': ['density_visceral_fat', 'density_muscle', 'density_bone', 'bmi']}


In [16]:
sfs_bwd_cardio = evaluate_multivariable_logit(sfs_bwd_cardio_features)
sfs_bwd_body = evaluate_multivariable_logit(sfs_bwd_body_features)

multivariable_results['sfs_bwd_cardio'] = sfs_bwd_cardio
multivariable_results['sfs_bwd_body'] = sfs_bwd_body

                                  Mean AUC   Std AUC
All Data                          0.766667       NaN
No Regularization, CV-10          0.691667  0.345708
L2 Regularization, CV-10          0.691667  0.345708
L1 Regularization, CV-10          0.658333  0.330088
ElasticNet Regularization, CV-10  0.691667  0.345708
                                            coef      pval  nobs signif
y           X                                                          
resolved_pe const                      -0.666636  0.121275    43       
            extrapulmonary_vein_volume -0.104176  0.761999    43       
            artery_vein_ratio           0.581935  0.145460    43       
            bv10                        0.011300  0.981747    43       
            pv_a                       -0.120208  0.820636    43       
            heart_volume                0.800663  0.047616    43      *
            airway_ratio                0.658196  0.170245    43       
            ild_volume            

### Sequential Feature Selection (Forward)

In [17]:
# Cardio
sfs_fwd_cardio_features = select_features_SFS(model=logit, scoring='roc_auc', features=cardio_features, target=logit_target, direction='forward')
print(sfs_fwd_cardio_features)

# Body
sfs_fwd_body_features = select_features_SFS(model=logit, scoring='roc_auc', features=body_features, target=logit_target, direction='forward')
print(sfs_fwd_body_features)

{'resolved_pe': ['heart_volume']}
{'resolved_pe': ['density_visceral_fat', 'density_bone']}


In [18]:
sfs_fwd_cardio = evaluate_multivariable_logit(sfs_fwd_cardio_features)
sfs_fwd_body = evaluate_multivariable_logit(sfs_fwd_body_features)

multivariable_results['sfs_fwd_cardio'] = sfs_fwd_cardio
multivariable_results['sfs_fwd_body'] = sfs_fwd_body

                                  Mean AUC   Std AUC
All Data                          0.680952       NaN
No Regularization, CV-10          0.750000  0.343592
L2 Regularization, CV-10          0.750000  0.343592
L1 Regularization, CV-10          0.750000  0.343592
ElasticNet Regularization, CV-10  0.750000  0.343592
                              coef      pval  nobs signif
y           X                                            
resolved_pe const        -0.754597  0.031069    43      *
            heart_volume  0.658342  0.053146    43       


                                  Mean AUC   Std AUC
All Data                          0.707143       NaN
No Regularization, CV-10          0.758333  0.284922
L2 Regularization, CV-10          0.758333  0.284922
L1 Regularization, CV-10          0.741667  0.294510
ElasticNet Regularization, CV-10  0.758333  0.284922
                                      coef      pval  nobs signif
y           X                                                   

### Export

In [19]:
metric_df = pd.DataFrame()

for key, result_dict in multivariable_results.items():
	temp_metric_df = result_dict['evaluation_metrics'].copy()
	feature_selection_method = [key] * temp_metric_df.shape[0]
	model_type = ['Logit'] * temp_metric_df.shape[0]
	temp_metric_df.index = pd.MultiIndex.from_tuples(zip(model_type, feature_selection_method, temp_metric_df.index))
	metric_df = pd.concat([metric_df, temp_metric_df], axis=0)

index_names = ['model_type', 'feature_selection_method', 'regularization']
metric_df = metric_df.reset_index(names=index_names)
metric_df.index = metric_df[index_names].apply('_'.join, axis=1)
metric_df.index.name = 'Lookup'

metric_df.to_csv('../output/regressions/logit_multi_metrics.csv')
metric_df.tail()

,model_type,feature_selection_method,regularization,Mean AUC,Std AUC
Lookup,,,,,
Logit_sfs_fwd_body_All Data,Logit,sfs_fwd_body,All Data,0.707143,NaN
"Logit_sfs_fwd_body_No Regularization, CV-10",Logit,sfs_fwd_body,"No Regularization, CV-10",0.758333,0.284922
"Logit_sfs_fwd_body_L2 Regularization, CV-10",Logit,sfs_fwd_body,"L2 Regularization, CV-10",0.758333,0.284922
"Logit_sfs_fwd_body_L1 Regularization, CV-10",Logit,sfs_fwd_body,"L1 Regularization, CV-10",0.741667,0.294510
"Logit_sfs_fwd_body_ElasticNet Regularization, CV-10",Logit,sfs_fwd_body,"ElasticNet Regularization, CV-10",0.758333,0.284922


In [20]:
regression_results_df = pd.DataFrame()

for key, result_dict in multivariable_results.items():
	temp_regression_results_df = result_dict['regression_results'].copy()
	feature_selection_method = [key] * temp_regression_results_df.shape[0]
	model_type = ['Logit'] * temp_regression_results_df.shape[0]
	temp_regression_results_df.index = pd.MultiIndex.from_tuples(
		zip(
			model_type, 
			feature_selection_method, 
			temp_regression_results_df.index.get_level_values(0),
			temp_regression_results_df.index.get_level_values(1),
		)
	)
	regression_results_df = pd.concat([regression_results_df, temp_regression_results_df], axis=0)
	
index_names = ['model_type', 'feature_selection_method', 'y_var', 'x_var']
regression_results_df = regression_results_df.reset_index(names=index_names)
regression_results_df.index = regression_results_df[index_names].apply('_'.join, axis=1)
regression_results_df.index.name = 'Lookup'

regression_results_df.to_csv('../output/regressions/logit_multi_regressions.csv')
regression_results_df.tail()

,model_type,feature_selection_method,y_var,x_var,coef,pval,nobs,signif
Lookup,,,,,,,,
Logit_sfs_fwd_cardio_resolved_pe_const,Logit,sfs_fwd_cardio,resolved_pe,const,-0.754597,0.031069,43,*
Logit_sfs_fwd_cardio_resolved_pe_heart_volume,Logit,sfs_fwd_cardio,resolved_pe,heart_volume,0.658342,0.053146,43,
Logit_sfs_fwd_body_resolved_pe_const,Logit,sfs_fwd_body,resolved_pe,const,-0.836079,0.017303,43,*
Logit_sfs_fwd_body_resolved_pe_density_visceral_fat,Logit,sfs_fwd_body,resolved_pe,density_visceral_fat,-0.363893,0.225374,43,
Logit_sfs_fwd_body_resolved_pe_density_bone,Logit,sfs_fwd_body,resolved_pe,density_bone,-0.703516,0.025257,43,*


# OLD